<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Part III - 3 <br><br><br>
  Chatbot with Memory
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

In [1]:
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
# package a installer d'abord avec anaconda
#import spacy

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# package a installer d'abord avec anaconda
import gensim
from gensim.models import KeyedVectors

#import nltk
#nltk.download()
#from nltk.tokenize import sent_tokenize, word_tokenize

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

import unidecode
from unidecode import unidecode

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
device = torch.device("cuda" if USE_CUDA else "cpu")

True


In [2]:
import sys
sys.path.append('C:\\Users\\Jb\\Desktop\\NLP\\DL4NLP lib')

In [4]:
path_to_NLP = 'C:\\Users\\Jb\\Desktop\\NLP'

# Corpus

Import a textfile containing dialogues and returns a list, each element 
corresponding to a dialogue and also being under the form of a list, with 
each element being a list of two elements : an element giving a user 
utterance and another element giving the bot response. Both elements are 
normalized strings. <br>
Exemple : The dialogue
       
               hi    hello what can i help you with today
               can you book a table    i m on it
               
now becomes
       
              [['hi', 'hello what can i help you with today'], 
               ['can you book a table', 'i m on it']]
               
Lines corresponding to user utterance with no bot response are discarted.

In [27]:
# --------------------------- Normalisation -------------------------------
def normalizeString(s):
    '''Remove rare symbols from a string'''
    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
 
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"[^a-zA-Z0-9?&\%\-\_]+", r" ", s) 
    s = re.sub("\(", r" ( ", s)
    s = re.sub("\)", r" ) ", s)
    s = re.sub(r"\.", r" . ", s)
    s = re.sub(r",", r" , ", s)
    s = re.sub(r"!", r" ! ", s)
    s = re.sub(r":", r" : ", s)
    s = re.sub(r"-", r" - ", s)
    s = re.sub(r"'", r" ' ", s)
    s = re.sub(r";", r" ; ", s)
    s = re.sub(r' +', r' ', s).strip()
    return s 



#--------------------- import des dialogues --------------------
def importDialogues(path, limit = None):
    '''Import a textfile containing dialogues and returns a list, each element 
       corresponding to a dialogue and also being under the form of a list, with 
       each element being a list of two elements : an element giving a user 
       utterance and another element giving the bot response. Both elements are 
       normalized strings.
       Ex. The dialogue :
       
               hi    hello what can i help you with today
               can you book a table    i m on it
               
       now becomes :
       
              [['hi', 'hello what can i help you with today'], 
               ['can you book a table', 'i m on it']]
               
       Lines corresponding to user utterance with no bot response are discarted.
    '''
    def cleanS(s):
        cleans = normalizeString(s)
        cleans = cleans.replace('?', ' ? ')
        cleans = cleans.split(' ')
        cleans = [word for word in cleans if word.strip() != '']
        return cleans
    
    dialogues = []
    dialogues_import = open(path, encoding='utf-8').read().strip().split('\n\n')
    for i, d in enumerate(dialogues_import):
        dialogue = []
        lines = d.split('\n')
        for l in lines:
            if len(l.split('\t')) == 2 :
                pair = [cleanS(s) for s in l.split('\t')]
                dialogue.append(pair)
            elif len(l.split('\t')) == 3 :
                pair = [cleanS(s) for s in l.split('\t')[:2]]
                dialogue.append(pair)
        dialogues.append(dialogue)
        if limit is not None and i == limit -1 : break
    return dialogues




#------------------ Dictionnaire des mots variables -----------------------------
def motVar(file):
    '''Applies to the Master's program dataset.
       Import the collection of pairs token-content for a set of variable words.
    '''
    lines = open(file, encoding='utf-8').read().strip().split('\n')
    motsVar = {}
    for l in lines :
        cle, valeur = l.split('\t')
        motsVar[cle.lower()] = valeur
    return motsVar

In [28]:
motsVar              = motVar(path_to_NLP + '\\data\\M2DS\\chatbot-M2-DS-Variables.txt')
dialogues_Master     = importDialogues(path_to_NLP + '\\data\\M2DS\\M2DS_2019_10\\ChatbotDS_P_Train.tsv')
dialogues_Master_tst = importDialogues(path_to_NLP + '\\data\\M2DS\\M2DS_2019_10\\ChatbotDS_P_Test.tsv')

In [29]:
for qa in dialogues_Master[0] : print(qa[0], '\t', qa[1])

['il', 'vaut', 'mieux', 'suivre', 'quel', 'parcours', '?'] 	 ['les', 'deux', 'parcours', 'sont', 'equivalents', ',', 'c', "'", 'est', 'a', 'vous', 'de', 'faire', 'un', 'choix', '.']
['quand', 'est', 'ce', 'que', 'commence', 'les', 'cours', '?'] 	 ['la', 'rentree', 'est', 'le', 'daterentree']
['comment', 'je', 'prend', 'contact', 'avec', 'la', 'scolarite', '?'] 	 ['les', 'contacts', 'de', 'la', 'scolarite', 'sont', 'accessibles', 'sur', ':', 'sitescolarite']
['est', 'ce', 'que', 'il', 'y', 'a', 'des', 'cours', 'en', 'anglais', '?'] 	 ['plusieurs', 'supports', 'de', 'cours', 'sont', 'en', 'anglais']
['tous', 'les', 'cours', 'sont', 'obligatoires', '?'] 	 ['tous', 'les', 'cours', 'sont', 'obligatoires', ',', 'il', 'n', "'", 'existe', 'pas', 'd', "'", 'options', '.']
['merci', 'pour', 'les', 'renseignements'] 	 ['je', 'vous', 'en', 'prie']
['peux', 'tu', 'm', "'", 'aider', '?'] 	 ['je', 'suis', 'la', 'pour', 'vous', 'renseigner', 'sur', 'le', 'master', '2', 'data', 'science', '.']
['le', '

## 1.3 Conversion in Torch variables

In [9]:
def variableFromSentence(lang, sentence, rand = 0): # sentence = [str]
    indices = []
    for word in sentence:
        p = random.random()
        if word not in lang.word2index.keys() and 'UNK' in lang.word2index.keys() :
            indices.append(lang.word2index['UNK'])
        elif p >= rand :
            indices.append(lang.word2index[word])
        elif p < rand :
            e = random.choice([1, 2])
            if e == 1 : pass # doesn't put any word
            elif e == 2 and 'UNK' in lang.word2index.keys() : indices.append(lang.word2index['UNK']) # hide word with UNK_Token
    indices.append(lang.word2index['EOS'])                                
    result = Variable(torch.LongTensor(indices)).view(-1, 1)
    return result

def variableFromDialogue(lang, dialogue, rand = 0): # sentence = [str]
    result = []
    for pair in dialogue :
        el1 = variableFromSentence(lang, pair[0], rand = rand)
        el2 = variableFromSentence(lang, pair[1], rand = 0)
        result.append([el1, el2])
    return result

def variableFromCorpus(lang, dialogues, rand = 0): # sentence = [str]
    return [variableFromDialogue(lang, dialogue, rand) for dialogue in dialogues]

# 1 Modules

## 1.1 Word Embedding module

[Back to top](#plan)

#### 1.1.1 FastText Embedding model

We consider here a FastText model trained following the Skip-Gram training objective.

In [7]:
from libDL4NLP.models.Word_Embedding import Word2Vec as myWord2Vec
from libDL4NLP.models.Word_Embedding import Word2VecConnector
from libDL4NLP.utils.Lang import Lang

In [9]:
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath, get_tmpfile

In [30]:
fastText_word2vec = FastText(size = 75, 
                             window = 5, 
                             min_count = 1, 
                             negative = 20,
                             sg = 1)

In [31]:
fastText_corpus = [['SOS'] + s + ['EOS'] for D in dialogues_Master for qa in D for s in qa]
len(fastText_corpus)

400448

In [32]:
fastText_word2vec.build_vocab(fastText_corpus)
len(fastText_word2vec.wv.vocab)

1288

In [33]:
fastText_word2vec.train(sentences = fastText_corpus, 
                        epochs = 5,
                        total_examples = fastText_word2vec.corpus_count)

In [38]:
file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\M2DS_2020_01 fasttext_75.model")
fastText_word2vec.save(file_name)

In [34]:
word2vec = Word2VecConnector(fastText_word2vec)

In [36]:
word2vec.twin.most_similar('inferentielle')

[('inferentielles', 0.79542387),
 ('factorielle', 0.6749466),
 ('profsi', 0.66358256),
 ('mining', 0.5842232),
 ('computing', 0.5814998),
 ('probabilistes', 0.57643235),
 ('graphiques', 0.56780875),
 ('statistique', 0.5646297),
 ('apprentissage', 0.53495073),
 ('learning', 0.53091097)]

## 1.2 Contextualization module

[Back to top](#plan)

The contextualization layer transforms a sequences of word vectors into another one, of same length, where each output vector corresponds to a new version of each input vector that is contextualized with respect to neighboring vectors.

<a id="bi_gru"></a>

#### 1.2.1 Bi-directionnal GRU contextualization

This module consists of a bi-directional _Gated Recurrent Unit_ (GRU) that supports packed sentences :

In [42]:
from libDL4NLP.modules import RecurrentEncoder

## 1.3 Attention module

[Back to top](#plan)

<a id="attention"></a>

#### 1.3.1 Classical Attention Module

In [43]:
from libDL4NLP.modules import Attention

#### 1.3.2 Hierarchical Attention Module

In [44]:
from libDL4NLP.modules import HAN

#### Visualisation of attention

In [ ]:
from libDL4NLP.utils import heatmap, annotate_heatmap

## 1.4 Word Decoding Module

In [ ]:
# TO revise
class WordsDecoder(nn.Module):
    '''Transforms a vector into a sequence of words'''
    def __init__(self, 
                 device, 
                 word2vec, 
                 hidden_dim, 
                 tracking_dim, 
                 dropout = 0.1,
                 tf_ratio = 1,
                 EOS_token = 1,
                 bound = 25
                ):
        super(WordsDecoder, self).__init__()
        # relevant quantities
        self.device = device
        self.hidden_dim = hidden_dim
        self.tracking_dim = tracking_dim
        self.tf_ratio = tf_ratio
        self.EOS_token = EOS_token
        self.bound = bound
        
        # modules
        self.word2vec = word2vec
        self.gru = nn.GRU(word2vec.lookupTable().shape[1] + tracking_dim, tracking_dim)
        self.out = nn.Linear(tracking_dim, word2vec.lookupTable().shape[0])
        self.dropout = nn.Dropout(dropout)
        
        
    def generateWord(self, query_vector, hidden, current_word_index):
        # update hidden state
        embedded = self.embedding(current_word_index).view(1, 1, -1)
        if query_vector is not None : embedded = torch.cat((query_vector, embedded), dim = 2)
        embedded = self.dropout(embedded)
        _, hidden = self.gru(embedded, hidden)
        # generate next word
        vector = self.out(hidden).squeeze(0)
        log_proba = F.log_softmax(vector, dim = 1)
        return log_proba, hidden
    
    
    def forward(self, query_words, query_vector, decision_vector, target_answer = None) :
        log_probas = []
        answer = []
        di = 0
        ta = target_answer if random.random() < self.tf_ratio else None
        current_word_index = Variable(torch.LongTensor([[0]])).to(self.device) # SOS_token
        hidden = self.dropout(decision_vector)
        for di in range(self.bound) :
            log_proba, hidden = self.generateWord(decision_vector, hidden, current_word_index) 
            log_probas.append(log_proba)
            ni = log_proba.data.topk(1)[1].item() # index of current generated word
            if ni == self.EOS_token : # EOS_token
                break
            elif ta is not None : # Teacher forcing
                answer.append(ni)
                if di < ta.size(0) : current_word_index = ta[di].to(self.device)
                else               : break
            else :
                answer.append(ni)
                current_word_index = Variable(torch.LongTensor([[ni]])).to(self.device)
        return answer, log_probas

***
# 2 Chatbot model

In [45]:
#from chatNLP.models.Chatbot import Chatbot, CreateBot, BotTrainer

In [ ]:
# TODO
class Chatbot(nn.Module):
    """Conversationnal agent with bi-GRU Encoder, taking as parameters at training time :
    
            -a complete dialogue of the form (with each content as string)
    
                    [['question 1', 'answer 1'],
                     ['question 2', 'answer 2'],
                             ..........
                     ['current question', 'current answer']]
     
            -the current answer for teacher forcing, or None
    
    and at test time :
    
            -the current question as string
    
    Returns :
     
            -word indices of the generated answer, according to output language of the model
            -attention weights of first attention layer, or None is no attention
            -attention weights of second attention layer, or None is no attention
    """
    def __init__(self, device, lang, encoder, attention, decoder, autoencoder = None):
        super(Chatbot, self).__init__()
        
        # relevant quantities
        self.lang = lang 
        self.device = device
        self.memory_dim = encoder.output_dim
        self.memory_length = 0
        
        # modules        
        self.encoder = encoder
        self.attention = attention
        self.decoder = decoder
        self.autoencoder = autoencoder
        
        
    # ---------------------- Technical methods -----------------------------
    def loadSubModule(self, encoder = None, attention = None, decoder = None) :
        if encoder is not None   : self.encoder = encoder
        if attention is not None : self.attention = attention
        if decoder is not None   : self.decoder = decoder
        return
    
    def freezeSubModule(self, encoder = False, attention = False, decoder = False) :
        for param in self.encoder.parameters()  : param.requires_grad = not encoder
        for param in self.attention.parameters(): param.requires_grad = not attention
        for param in self.decoder.parameters()  : param.requires_grad = not decoder
        return
    
    def nbParametres(self) :
        return sum([p.data.nelement() for p in self.parameters() if p.requires_grad == True])
    
    def flatten(self, description):
        '''Baisse le nombre de niveaux de 1 dans la description'''
        flatten = []
        for line in description :
            if type(line) == list : flatten += line  
            else                  : flatten.append(line)
        return [[int(word) for word in sentence.data.view(-1)] for sentence in flatten]

    
    
    # ------------------------ Text processing methods ---------------------------------
    def variableFromSentence(self, sentence):
        def normalizeString(sentence) :
            '''Remove rare symbols from a string'''
            def unicodeToAscii(s):
                """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
                return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
            sentence = unicodeToAscii(sentence.lower().strip())
            sentence = re.sub(r"[^a-zA-Z0-9?&\%\-\_]+", r" ", sentence) 
            return sentence
        sentence = normalizeString(sentence).split(' ') # a raw string transformed into a list of clean words
        indexes = []
        #unknowns = 0
        for word in sentence:
            if word not in self.lang.word2index.keys() :
                if 'UNK' in self.lang.word2index.keys() : indexes.append(self.lang.word2index['UNK'])
            else :
                indexes.append(self.lang.word2index[word])
        #indexes.append(self.lang.word2index['EOS']) 
        indexes.append(1) # EOS_token
        result = Variable(torch.LongTensor([[i] for i in indexes]))
        return result
    
    
    
    # ------------------------ Visualisation methods ---------------------------------
    def flattenDialogue(self, dialogue):
        flatten = []
        for paire in dialogue : flatten += paire
        return [[int(word) for word in sentence.data.view(-1)] for sentence in flatten]
    
    def flattenWeights(self, weights) :
        '''Baisse le nombre de niveaux de 1 dans les poids d'attention'''
        flatten = []
        for weight_layer in weights : flatten.append(torch.cat(tuple(weight_layer.values()), dim = 2))
        return flatten
    
    def formatWeights(self, dialogue, attn1_weights, attn2_weights) :
        if self.n_level == 2 : attn1_weights = self.flattenWeights(attn1_weights)
        hops = self.attention.hops
        l, L = len(dialogue), max([len(line) for line in dialogue])
        Table = np.zeros((l, 1, L))
        Liste = np.zeros((l, 1)) if attn2_weights is not None else None
        count = 0
        count_line = 0
        for i, line in enumerate(dialogue) :
            present = False
            for j, word in enumerate(line) :
                if word in self.lang.index2word.keys():
                    present = True
                    Table[i, 0, j] = sum([attn1_weights[k][0, 0, count].data for k in range(hops)])
                    count += 1
            if present and Liste is not None :
                Liste[i] = sum([attn2_weights[k][count_line].data for k in range(hops)])
                count_line += 1
        return Table, Liste
    
    def showWeights(self, dialogue, attn1_weights, attn2_weights, maxi):
        table, liste = self.formatWeights(dialogue[:-2], attn1_weights, attn2_weights)
        l = table.shape[0]
        L = table.shape[2]
        fig = plt.figure(figsize = (l, L))
        for i, line in enumerate(dialogue[:-2]):
            ligne = [self.lang.index2word[int(word)] for word in line]
            ax = fig.add_subplot(l, 1, i+1)
            vals = table[i]
            text = [' '] + ligne + [' ' for k in range(L-len(ligne))] if L>len(ligne) else [' '] + ligne
            if liste is not None :
                vals = np.concatenate((np.zeros((1, 1)) , vals), axis = 1)  
                vals = np.concatenate((np.reshape(liste[i], (1, 1)) , vals), axis = 1)
                turn = 'User' if i % 2 == 0 else 'Bot'
                text = [turn] + [' '] + text
            cax = ax.matshow(vals, vmin=0, vmax=maxi, cmap='YlOrBr')
            ax.set_xticklabels(text, ha='left')
            ax.set_yticklabels(' ')
            ax.tick_params(axis=u'both', which=u'both',length=0, labelrotation = 30, labelright  = True)
            ax.grid(b = False, which="minor", color="w", linestyle='-', linewidth=1)
            ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
            plt.subplots_adjust(hspace=0, wspace = 0.1)
        plt.show()
    
    def showAttention(self, dialogue, n_col = 1, maxi = None):
        answer, decoder_outputs, attn1_w, attn2_w, _ = self.answerTrain(dialogue)
        dialogue = self.flattenDialogue(dialogue)
        if len(dialogue) > 1 : self.showWeights(dialogue, attn1_w, attn2_w, maxi)
        print('User : ', ' '.join([self.lang.index2word[int(word)] for word in dialogue[-2][:-1]]))
        print('target : ', ' '.join([self.lang.index2word[int(word)] for word in dialogue[-1][:-1]]))
        print('predic : ', ' '.join([self.lang.index2word[int(word)] for word in answer]))
        return
    
    
    
    # ----------------------- Memory ----------------------------
    def initMemory(self):
        """Initialize memory slots"""
        self.memory = {}
        self.memory_queries = {}
        self.query_hidden = None
        self.memory_length = 0
        
    def updateMemory(self, last_words, query_hidden):
        """Update memory with a list of word vectors 'last_words' and the last query vector 'last_query'"""
        self.memory[self.memory_length] = last_words
        self.memory_queries[self.memory_length] = query_hidden
        self.query_hidden = query_hidden
        self.memory_length += 1
        
        
    # ------------------- Process methods ------------------------
    def readSentence(self, utterance):
        """Perform reading of an utterance, returning created word vectors
           and last hidden states of teh encoder bi-GRU
        """
        utterance = utterance.to(self.device)
        last_words, query_hidden = self.encoder(utterance, self.query_hidden)
        return last_words, query_hidden
        
    def readDialogue(self, dialogue):
        """Loop of readUtterance over a whole dialogue
        """
        for i in range(len(dialogue)) :
            if type(dialogue[i]) == list :
                for utterance in dialogue[i]:
                    last_words, query_hidden = self.readSentence(utterance)
                    self.updateMemory(last_words, query_hidden)
            else :
                utterance = dialogue[i]
                last_words, query_hidden = self.readSentence(utterance)
                self.updateMemory(last_words, query_hidden)
   
    def tracking(self, query_vector = None):
        """Détermine un vecteur d'attention sur les éléments du registre de l'agent,
        sachant un vecteur 'very_last_hidden', et l'accole à ce vecteur """
        decision_vector, attn1_weights, attn2_weights = self.attention(words_memory = self.memory, 
                                                                       query = query_vector)
        return decision_vector, attn1_weights, attn2_weights

    def generateAnswer(self,last_words, query_vector, decision_vector, target_answer = None) :
        """Génère une réponse à partir d'un état caché initialisant le décodeur,
        en utilisant une réponse cible pour un mode 'teacher forcing-like' si celle-ci est fournie """
        answer, decoder_outputs = self.decoder(last_words, query_vector, decision_vector, target_answer)
        return answer, decoder_outputs
    
    def generateQuery(self,last_words, query_vector, decision_vector, target_answer = None) :
        """Génère une réponse à partir d'un état caché initialisant le décodeur,
        en utilisant une réponse cible pour un mode 'teacher forcing-like' si celle-ci est fournie """
        if self.autoencoder is not None : 
            query, autoencoder_outputs = self.autoencoder(last_words, query_vector, decision_vector, target_answer)
            return query, autoencoder_outputs
        else :
            return None, None
        
        
        
    # ------------ 1st working mode : training mode ------------
    def answerTrain(self, input, target_answer = None):
        """Parameters are a complete dialogue, containing the current query,
           
           - either of the form :

                    [['query 1', 'answer 1'],
                     ['query 2', 'answer 2'],
                             ..........
                     ['current query', 'current answer']]
                     
           - or :
           
                    ['query 1',
                     'query 2',
                       ....
                     'current query'] 

           The model learns to generate the current answer. 
           Teacher forcing can be enabled by passing the ground answer though the 'target_answer' option.
        """
        # 1) initiates memory instance
        self.initMemory()
        
        # 2) reads historical part of dialogue (if applicable),
        # word vectors and last hidden states of encoder bi-GRU are stored in memory
        dialogue = input[:-1]
        self.readDialogue(dialogue)
        
        # 3) reads current utterance,
        # returns word vectors of query and query vector
        query = input[-1]
        query = query[0] if type(query) == list else query
        last_words, query_hidden = self.readSentence(query)
        query_hidden = query_hidden.view(1, 1, -1)
        
        # 4) performs tracking
        # returns decision vector
        if self.attention is not None :
            decision_vector, attn1_weights, attn2_weights = self.tracking(query_hidden)
        else :
            decision_vector = query_hidden
            attn1_attention_weights = None
            attn2_attention_weights = None
            
        # 5) response generation
        # returns list of indices
        answer, decoder_outputs = self.generateAnswer(last_words, query_hidden, decision_vector, target_answer)
        pred_query, autoencoder_outputs = self.generateQuery(last_words, query_hidden, decision_vector, target_answer)    
        # 6) returns answer
        return answer, decoder_outputs, attn1_weights, attn2_weights, autoencoder_outputs

        
        
    # ------------ 2nd working mode : test mode ------------
    def forward(self, input):
        """Parameters are a single current query as string, and the model attempts to generate the current answer.
        """
        
        # 1) initiates memory and hidden states of encoder bi-GRU if conversation starts
        if self.memory_length == 0 : self.initMemory()
            
        # 2) reads current utterance,
        # returns word vectors of query and query vector
        sentence = self.variableFromSentence(input)
        if sentence is None :
            return "Excusez-moi je n'ai pas compris", None, None
        else :
            last_words, query_hidden = self.readSentence(sentence)
            q_hidden = query_hidden.view(1, 1, -1)

            # 3) performs tracking
            # returns decision vector
            if self.attention is not None :
                decision_vector, attn1_weights, attn2_weights = self.tracking(q_hidden)
            else :
                decision_vector = q_hidden
                attn1_attention_weights = None
                attn2_attention_weights = None

            # 4) response generation
            # returns list of indices
            answer, decoder_outputs = self.generateAnswer(last_words, q_hidden, decision_vector)
            
            # 5) updates memory with current query and answer
            self.updateMemory(last_words, query_hidden)
            answer_var = Variable(torch.LongTensor([[i] for i in answer]))
            last_words, query_hidden = self.readSentence(answer_var)
            self.updateMemory(last_words, query_hidden)

            # 6) returns answer
            answer = ' '.join([self.lang.index2word[int(word)] for word in answer])
            return answer, attn1_weights, attn2_weights

In [ ]:
#-------------------------------------------------------------------#
#                         Instanciator                              #
#-------------------------------------------------------------------#
    
    
# TODO    
def CreateBot(lang,                     ###
              embedding,                  # --- Encoder options
              hidden_dim,                 #
              n_layers,                 ###

              sentence_hidden_dim,      ###
              hops,                       #
              share,                      # --- Hierarchical encoder options
              transf,                     #
              dropout,                  ###
              
              attn_decoder_n_layers,    ###
              language_model_n_layers,    #
              tf_ratio,                   # --- decoder options
              bound,                      #
              autoencoding,             ###
              
              device
             ):
    '''Create an agent with specified dimensions and specificities'''
    # 1) ----- encoding -----
    EOS_token = lang.word2index['EOS'] if 'EOS' in lang.word2index.keys() else 1
    if type(embedding) == torch.nn.modules.sparse.Embedding : 
        for param in embedding.parameters() : param.requires_grad = False
    elif type(embedding) == int : 
        embedding = nn.Embedding(lang.n_words, embedding) 
    else : 
        embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding), freeze=True)

    encoder = RecurrentWordsEncoder(device, 
                                    embedding, 
                                    hidden_dim, 
                                    n_layers, 
                                    dropout) # embedding, hidden_dim, n_layers = 1, dropout = 0
    # 2) ----- attention -----
    word_hidden_dim = encoder.output_dim
    attention = RecurrentHierarchicalAttention(device,
                                               word_hidden_dim,
                                               sentence_hidden_dim, 
                                               query_dim = word_hidden_dim,
                                               n_heads = 1,
                                               n_layers = n_layers,
                                               hops = hops,
                                               share = share,
                                               transf = transf,
                                               dropout = dropout)
    # 3) ----- decoding -----
    tracking_dim = attention.output_dim
    autoencoder = None
    if language_model_n_layers > 0 :
        language_model = LanguageModel(device, 
                                       lang,
                                       embedding = embedding, 
                                       hidden_dim = hidden_dim,
                                       n_layers = language_model_n_layers, 
                                       dropout = dropout)
        decoder = LMWordsDecoder(device,
                                 language_model,                                   
                                 word_hidden_dim,
                                 tracking_dim,
                                 dropout = dropout,
                                 tf_ratio = tf_ratio,
                                 bound = bound)
        if autoencoding :
            autoencoder = LMWordsDecoder(device,
                                         language_model,                                   
                                         word_hidden_dim,
                                         tracking_dim,
                                         dropout = dropout,
                                         tf_ratio = tf_ratio,
                                         bound = bound)
        
    elif attn_decoder_n_layers >= 0 :
        decoder = AttnWordsDecoder(device,
                                   embedding,
                                   word_hidden_dim,
                                   tracking_dim,
                                   dropout = dropout,
                                   n_layers = attn_decoder_n_layers,
                                   tf_ratio = tf_ratio,
                                   bound = bound)
        if autoencoding :
            autoencoder = AttnWordsDecoder(device,
                                           embedding,
                                           word_hidden_dim,
                                           tracking_dim,
                                           dropout = dropout,
                                           n_layers = attn_decoder_n_layers,
                                           tf_ratio = tf_ratio,
                                           bound = bound)
    else :
        decoder = WordsDecoder(device,
                               embedding,                                   
                               word_hidden_dim,
                               tracking_dim,
                               dropout = dropout,
                               tf_ratio = tf_ratio,
                               EOS_token = EOS_token,
                               bound = bound)       
        if autoencoding :
            autoencoder = WordsDecoder(device,
                                       embedding,                                   
                                       word_hidden_dim,
                                       tracking_dim,
                                       dropout = dropout,
                                       tf_ratio = tf_ratio,
                                       EOS_token = EOS_token,
                                       bound = bound) 
    # 4) ----- model -----
    chatbot = Chatbot(device, lang, encoder, attention, decoder, autoencoder)
    chatbot = chatbot.to(device)
    return chatbot

In [ ]:
#-------------------------------------------------------------------#
#                             Trainer                               #
#-------------------------------------------------------------------#

# TODO
class BotTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(), 
                 optimizer = optim.SGD, 
                 clipping = 10, 
                 print_every=100):
        
        # relevant quantities
        self.device = device
        self.criterion = criterion.to(device)
        self.optimizer = optimizer
        self.clip = clipping
        self.print_every = print_every# timer
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def distance(self, agent_outputs, target_answer) :
        """ Compute cumulated error between predicted output and ground answer."""
        loss = 0
        loss_diff_mots = 0
        agent_outputs_length = len(agent_outputs)
        target_length = len(target_answer)
        Max = max(agent_outputs_length, target_length)
        Min = min(agent_outputs_length, target_length)   
        for i in range(Min):
            agent_output = agent_outputs[i]
            target_word = target_answer[i]
            #print(agent_output.size(), target_answer.size())
            factor = (1 + Max - Min) if i == Min -1 else 1
            loss += factor * self.criterion(agent_output, target_word)
            topv, topi = agent_output.data.topk(1)
            ni = topi[0][0]
            if ni != target_word.data[0]:
                loss_diff_mots += 1
        loss_diff_mots += Max - Min
        return loss, loss_diff_mots
        
        
    def trainLoop(self, agent, dialogue, target_answer, optimizer):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        query = dialogue[-1][0].to(self.device)
        target_answer = target_answer.to(self.device)
        answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue, target_answer) 
        loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
        loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(agent.parameters(), self.clip)
        optimizer.step()
        return loss.data[0] / len(target_answer), loss_diff_mots
        
        
    def train(self, agent, dialogues, n_iters = 10000, learning_rate=0.01, dic = None, per_dialogue = False, return_errors = False):
        """Performs training over a given dataset and along a specified amount of loops."""
        if type(dialogues[0][0]) == list :
            debut = 0
            double = True
        else :
            debut = 1
            double = False
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if return_errors : errors = {}
        iter = 1
        while iter < n_iters :
            if dic is not None :
                j = int(random.choice(list(dic.keys())))
                training_dialogue = dialogues[j]
                i = random.choice(dic[j])
                partie_dialogue = training_dialogue[:i+1-debut]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                if return_errors and loss_diff_mots > 0 :
                    if j not in list(errors.keys()) : errors[j] = []
                    if i not in errors[j] : errors[j].append(i)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots 
                iter += 1
                if iter % self.print_every == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                  print_loss_avg, print_loss_diff_mots_avg))
            elif per_dialogue :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                indices = list(range(debut, len(training_dialogue)))
                random.shuffle(indices)
                for i in indices :
                    partie_dialogue = training_dialogue[:i+1]
                    target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                    loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                    if return_errors and loss_diff_mots > 0 :
                        if j not in list(errors.keys()) : errors[j] = []
                        if i not in errors[j] : errors[j].append(i)
                    # quantité d'erreurs sur la réponse i
                    print_loss_total += loss
                    print_loss_diff_mots_total += loss_diff_mots
                    iter += 1
                    if iter % self.print_every == 0:
                        print_loss_avg = print_loss_total / self.print_every
                        print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                        print_loss_total = 0
                        print_loss_diff_mots_total = 0
                        print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                      print_loss_avg, print_loss_diff_mots_avg))
            else :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                i = random.choice(range(debut, len(training_dialogue)))
                partie_dialogue = training_dialogue[:i+1]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                if return_errors and loss_diff_mots > 0 :
                    if j not in list(errors.keys()) : errors[j] = []
                    if i not in errors[j] : errors[j].append(i)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots
                iter += 1
                if iter % self.print_every == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                  print_loss_avg, print_loss_diff_mots_avg))


        if return_errors : return errors
                
                
    def ErrorCount(self, agent, dialogues):
        bound = 10
        ERRORS = [0 for i in range(bound +1)]
        repartitionError = {}
        for i in range(bound +1) :
            repartitionError[i] = []
        liste = []
        for k, input_dialogue in enumerate(dialogues):
            for l in range(len(input_dialogue)):
                if len(input_dialogue[l][1])>0 :
                    dialogue = input_dialogue[:l+1]
                    #target_answer = variableFromSentence(agent.output_lang, input_dialogue[l][1])
                    target_answer = input_dialogue[l][1]
                    target_answer = target_answer.to(self.device)
                    answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue)
                    loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                    if loss_diff_mots > bound :
                        ERRORS = ERRORS + [0 for i in range(loss_diff_mots - bound)]
                        for i in range(bound +1, loss_diff_mots +1) :
                            repartitionError[i] = []
                        bound  = loss_diff_mots
                    ERRORS[loss_diff_mots] += 1
                    if loss_diff_mots > 0 :
                        liste.append([k, l, loss_diff_mots])
        for triple in liste:
            repartitionError[triple[2]].append(triple[:2])
        print("The repartition of errors :", ERRORS)
        return repartitionError


    def DialoguesWithErrors(self, agent, dialogues) :
        '''Returns a dictionnary, with indices of dialogues and index of line in dialogue
           where a mistake was made.
        '''
        start = time.time()
        Sortie = {}
        L = len(dialogues)
        for i, dialogue in enumerate(dialogues) :
            errs = []
            for j in range(len(dialogue)) :
                target_answer = dialogue[j][1]
                target_answer = target_answer.to(self.device)
                answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue[:j+1], target_answer)
                loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                if loss_diff_mots > 0 :
                    errs.append(j)
            if errs != []:
                Sortie[i] = errs
            if (i+1) % self.print_every == 0:
                print('%s (%d %d%%)' % (self.timeSince(start, (i+1) / L),
                                             (i+1), (i+1) / L * 100))
        return Sortie

# 3 Training

## 3.1 Data

In [46]:
dialogues_Master_var = variableFromCorpus(lang_M2DS, dialogues_Master, rand = 0)

In [47]:
dialogues_Master_test_var = variableFromCorpus(lang_M2DS, dialogues_Master_tst, rand = 0)

In [48]:
bound = max([max([len(el[1]) for el in dial]) for dial in dialogues_Master_var])
bound

70

## 6.2 Model

In [49]:
trainer = BotTrainer(device = device,
                     criterion = nn.NLLLoss(), 
                     optimizer = optim.SGD,
                     #autoencoding_ratio = 0.5,
                     clipping = 10,
                     print_every = 100)

In [50]:
chatbot = CreateBot(  lang_M2DS,
                      embedding = skipgram.embedding, #fastText_gensim.wv.vectors, 
                      hidden_dim = 75,
                      n_layers = 1, # doit être gardé à 1
                    
                      sentence_hidden_dim = 75,
                      hops = 3,
                      share = True, #False
                      transf = True,
                      dropout = 0.10,
                    
                      attn_decoder_n_layers = -1, # -1 : pas d'attention durant le décodage
                      language_model_n_layers = 0,
                      tf_ratio = 1,
                      bound = 75,
                      autoencoding = False,
                    
                      device = device
                     )

chatbot.nbParametres()

649147

In [22]:
chatbot.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_3.pth')) # chatbot_original / 26 / 39

In [51]:
# si on veut voir le pipeline du modèle
chatbot

Chatbot(
  (encoder): RecurrentWordsEncoder(
    (embedding): Embedding(1297, 75)
    (dropout): Dropout(p=0.1)
    (bigru): GRU(75, 75, bidirectional=True)
  )
  (attention): RecurrentHierarchicalAttention(
    (dropout): Dropout(p=0.1)
    (attn1): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
      (1): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
      (2): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
    )
    (bigru): GRU(150, 75, bidirectional=True)
    (attn2): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
      

## 6.3 Training

### 6.3.1 Standard training

In [98]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05)

0m 19s (- 64m 27s) (100 0%) 5.9824 15.26
0m 44s (- 73m 46s) (200 1%) 5.4691 15.16
1m 10s (- 77m 13s) (300 1%) 4.6947 14.88
1m 32s (- 75m 13s) (400 2%) 3.7345 11.41
1m 54s (- 74m 11s) (500 2%) 3.5334 11.05
2m 14s (- 72m 26s) (600 3%) 3.1608 9.46
2m 35s (- 71m 16s) (700 3%) 2.5577 8.05
2m 52s (- 68m 58s) (800 4%) 2.7213 9.14
3m 10s (- 67m 20s) (900 4%) 3.0129 9.85
3m 30s (- 66m 48s) (1000 5%) 2.4027 8.41
3m 54s (- 67m 7s) (1100 5%) 2.5969 8.01
4m 11s (- 65m 34s) (1200 6%) 1.9883 6.17
4m 32s (- 65m 17s) (1300 6%) 2.0248 6.73
4m 56s (- 65m 44s) (1400 7%) 1.9121 6.53
5m 17s (- 65m 11s) (1500 7%) 1.5863 6.49
5m 40s (- 65m 16s) (1600 8%) 2.0049 7.08
6m 2s (- 65m 4s) (1700 8%) 2.0101 6.34
6m 28s (- 65m 24s) (1800 9%) 1.7517 5.32
6m 55s (- 65m 53s) (1900 9%) 2.1832 7.18
7m 21s (- 66m 16s) (2000 10%) 1.9682 9.56
7m 41s (- 65m 36s) (2100 10%) 1.6043 5.99
8m 7s (- 65m 40s) (2200 11%) 1.4449 5.00
8m 29s (- 65m 19s) (2300 11%) 1.5355 5.02
8m 51s (- 65m 1s) (2400 12%) 1.3743 6.31
9m 9s (- 64m 6s) (25

70m 50s (- 2m 34s) (19300 96%) 0.2659 1.26
71m 11s (- 2m 12s) (19400 97%) 0.0909 0.30
71m 35s (- 1m 50s) (19500 97%) 0.1567 0.51
71m 55s (- 1m 28s) (19600 98%) 0.1800 0.56
72m 18s (- 1m 6s) (19700 98%) 0.1531 0.48
72m 37s (- 0m 44s) (19800 99%) 0.1102 0.42
72m 59s (- 0m 22s) (19900 99%) 0.2055 1.32
73m 23s (- 0m 0s) (20000 100%) 0.0994 0.28


In [99]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_1.pth')

In [100]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05)

0m 17s (- 58m 12s) (100 0%) 0.1849 0.63
0m 40s (- 66m 38s) (200 1%) 0.1055 0.40
0m 58s (- 64m 9s) (300 1%) 0.1193 0.45
1m 21s (- 66m 39s) (400 2%) 0.1358 0.44
1m 39s (- 64m 46s) (500 2%) 0.0835 0.28
1m 58s (- 63m 49s) (600 3%) 0.1094 0.36
2m 15s (- 62m 28s) (700 3%) 0.1917 0.72
2m 35s (- 62m 4s) (800 4%) 0.1567 0.51
2m 55s (- 62m 7s) (900 4%) 0.1496 0.40
3m 17s (- 62m 37s) (1000 5%) 0.1340 0.48
3m 38s (- 62m 29s) (1100 5%) 0.1194 0.34
4m 2s (- 63m 18s) (1200 6%) 0.0837 0.35
4m 24s (- 63m 20s) (1300 6%) 0.1436 0.69
4m 42s (- 62m 39s) (1400 7%) 0.1104 0.37
5m 4s (- 62m 36s) (1500 7%) 0.1358 0.52
5m 24s (- 62m 14s) (1600 8%) 0.1801 0.64
5m 49s (- 62m 40s) (1700 8%) 0.1816 0.61
6m 13s (- 62m 58s) (1800 9%) 0.2046 1.53
6m 35s (- 62m 44s) (1900 9%) 0.1485 0.45
6m 59s (- 62m 57s) (2000 10%) 0.1426 0.66
7m 22s (- 62m 52s) (2100 10%) 0.1441 0.69
7m 42s (- 62m 22s) (2200 11%) 0.1476 0.64
8m 4s (- 62m 9s) (2300 11%) 0.1164 0.50
8m 28s (- 62m 7s) (2400 12%) 0.1801 1.00
8m 54s (- 62m 20s) (2500 12%

70m 24s (- 2m 33s) (19300 96%) 0.0510 0.14
70m 46s (- 2m 11s) (19400 97%) 0.1046 0.69
71m 10s (- 1m 49s) (19500 97%) 0.0371 0.14
71m 30s (- 1m 27s) (19600 98%) 0.0274 0.11
71m 55s (- 1m 5s) (19700 98%) 0.0765 0.25
72m 21s (- 0m 43s) (19800 99%) 0.0514 0.17
72m 41s (- 0m 21s) (19900 99%) 0.0405 0.14
73m 6s (- 0m 0s) (20000 100%) 0.0278 0.08


In [101]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_1.pth')

### 6.3.2 Teacher Forcing Ratio = 0.5

In [102]:
chatbot.decoder.tf_ratio = 0.5

In [103]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05)

0m 19s (- 66m 3s) (100 0%) 1.1763 1.10
0m 43s (- 71m 50s) (200 1%) 0.6570 0.60
1m 1s (- 66m 50s) (300 1%) 1.3066 0.98
1m 25s (- 70m 10s) (400 2%) 1.1022 1.24
1m 48s (- 70m 50s) (500 2%) 0.6232 0.58
2m 13s (- 71m 41s) (600 3%) 0.5726 0.68
2m 35s (- 71m 34s) (700 3%) 0.8481 0.98
3m 0s (- 72m 23s) (800 4%) 0.9288 1.39
3m 21s (- 71m 11s) (900 4%) 1.1647 1.30
3m 46s (- 71m 37s) (1000 5%) 1.2660 1.72
4m 5s (- 70m 20s) (1100 5%) 0.9216 1.32
4m 27s (- 69m 49s) (1200 6%) 1.0929 1.48
4m 48s (- 69m 8s) (1300 6%) 1.4344 1.83
5m 9s (- 68m 35s) (1400 7%) 1.6573 2.05
5m 25s (- 66m 58s) (1500 7%) 0.3042 0.55
5m 49s (- 66m 55s) (1600 8%) 0.7867 0.80
6m 10s (- 66m 33s) (1700 8%) 1.2536 1.98
6m 34s (- 66m 24s) (1800 9%) 1.3726 2.47
6m 53s (- 65m 38s) (1900 9%) 0.8000 0.89
7m 14s (- 65m 7s) (2000 10%) 0.8321 0.91
7m 41s (- 65m 31s) (2100 10%) 0.5531 1.06
8m 6s (- 65m 33s) (2200 11%) 0.5990 0.91
8m 25s (- 64m 46s) (2300 11%) 0.8929 1.54
8m 48s (- 64m 38s) (2400 12%) 0.6706 0.89
9m 13s (- 64m 32s) (2500 12%

72m 7s (- 2m 36s) (19300 96%) 0.4287 0.52
72m 29s (- 2m 14s) (19400 97%) 0.7204 0.82
72m 46s (- 1m 51s) (19500 97%) 0.5925 1.12
73m 13s (- 1m 29s) (19600 98%) 0.4583 0.71
73m 38s (- 1m 7s) (19700 98%) 0.3996 0.66
74m 4s (- 0m 44s) (19800 99%) 0.1390 0.21
74m 27s (- 0m 22s) (19900 99%) 0.4688 0.44
74m 55s (- 0m 0s) (20000 100%) 0.4470 1.11


In [104]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_2.pth')

### 6.3.3 Random masking 25 % of corpus words

In [105]:
dialogues_Master_var_025 = variableFromCorpus(lang_M2DS, dialogues_Master, rand = 0.25)

In [106]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 30000, per_dialogue = True, learning_rate=0.05)

0m 23s (- 115m 4s) (100 0%) 2.8841 4.68
0m 42s (- 105m 44s) (200 0%) 3.4752 4.50
1m 3s (- 104m 18s) (300 1%) 3.3769 4.00
1m 23s (- 102m 32s) (400 1%) 4.7090 5.15
1m 47s (- 105m 55s) (500 1%) 2.5536 4.78
2m 7s (- 104m 8s) (600 2%) 3.3206 4.31
2m 29s (- 104m 16s) (700 2%) 3.0950 4.71
2m 47s (- 102m 8s) (800 2%) 2.9677 3.56
3m 8s (- 101m 43s) (900 3%) 1.8963 3.52
3m 31s (- 102m 26s) (1000 3%) 2.9690 4.62
3m 51s (- 101m 25s) (1100 3%) 1.7392 2.99
4m 15s (- 102m 19s) (1200 4%) 3.0840 4.17
4m 38s (- 102m 20s) (1300 4%) 2.6181 3.72
5m 1s (- 102m 40s) (1400 4%) 2.5398 4.06
5m 22s (- 102m 12s) (1500 5%) 2.5670 3.66
5m 48s (- 103m 6s) (1600 5%) 1.6853 2.70
6m 8s (- 102m 21s) (1700 5%) 2.8136 3.92
6m 29s (- 101m 34s) (1800 6%) 2.1147 3.53
6m 51s (- 101m 22s) (1900 6%) 3.2852 5.53
7m 13s (- 101m 13s) (2000 6%) 2.5784 3.79
7m 35s (- 100m 47s) (2100 7%) 2.0950 3.18
7m 54s (- 100m 0s) (2200 7%) 2.6686 4.68
8m 18s (- 100m 5s) (2300 7%) 3.4213 5.71
8m 40s (- 99m 47s) (2400 8%) 2.5750 3.70
8m 57s (- 98m

642m 46s (- 372m 8s) (19000 63%) 1.5818 3.47
643m 2s (- 366m 58s) (19100 63%) 2.0521 3.22
643m 27s (- 361m 56s) (19200 64%) 1.7678 3.40
643m 46s (- 356m 54s) (19300 64%) 1.5953 3.00
644m 12s (- 351m 59s) (19400 64%) 2.1417 2.54
644m 32s (- 347m 3s) (19500 65%) 2.2544 3.78
644m 59s (- 342m 14s) (19600 65%) 1.7166 3.45
645m 25s (- 337m 27s) (19700 65%) 2.4452 4.23
645m 46s (- 332m 40s) (19800 66%) 1.4094 2.65
646m 11s (- 327m 58s) (19900 66%) 2.2573 3.03
646m 35s (- 323m 17s) (20000 66%) 2.3647 3.76
646m 53s (- 318m 36s) (20100 67%) 1.6359 2.62
647m 12s (- 313m 59s) (20200 67%) 1.8993 3.13
647m 28s (- 309m 23s) (20300 67%) 1.7366 3.72
647m 49s (- 304m 51s) (20400 68%) 1.4981 2.84
648m 15s (- 300m 24s) (20500 68%) 2.7461 4.31
648m 39s (- 295m 59s) (20600 68%) 3.0859 4.97
648m 58s (- 291m 34s) (20700 69%) 1.6551 3.25
649m 20s (- 287m 12s) (20800 69%) 2.2488 4.45
649m 40s (- 282m 52s) (20900 69%) 1.4532 2.59
650m 3s (- 278m 35s) (21000 70%) 0.7888 1.89
650m 25s (- 274m 21s) (21100 70%) 2.09

In [107]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_3.pth')

### 6.3.3 Learning rate harved

In [108]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 30000, per_dialogue = True, learning_rate=0.025)

0m 20s (- 102m 47s) (100 0%) 1.1408 3.19
0m 44s (- 109m 53s) (200 0%) 2.5181 5.22
1m 6s (- 109m 23s) (300 1%) 1.2684 3.03
1m 30s (- 111m 10s) (400 1%) 1.4725 3.05
1m 50s (- 109m 0s) (500 1%) 2.1122 3.63
2m 13s (- 108m 56s) (600 2%) 1.2463 2.95
2m 40s (- 111m 52s) (700 2%) 2.2863 5.22
2m 58s (- 108m 50s) (800 2%) 1.1197 2.08
3m 17s (- 106m 21s) (900 3%) 1.2375 3.17
3m 41s (- 107m 12s) (1000 3%) 1.6045 2.74
4m 2s (- 106m 24s) (1100 3%) 0.9433 2.30
4m 24s (- 105m 36s) (1200 4%) 1.9456 3.62
4m 44s (- 104m 30s) (1300 4%) 1.6842 2.96
5m 9s (- 105m 14s) (1400 4%) 1.4658 2.54
5m 32s (- 105m 14s) (1500 5%) 1.9937 3.17
5m 53s (- 104m 27s) (1600 5%) 1.4611 2.79
6m 16s (- 104m 24s) (1700 5%) 2.5454 3.92
6m 41s (- 104m 48s) (1800 6%) 1.5519 3.28
7m 3s (- 104m 20s) (1900 6%) 1.8223 3.69
7m 30s (- 105m 0s) (2000 6%) 1.7267 3.66
7m 50s (- 104m 13s) (2100 7%) 1.1564 2.54
8m 15s (- 104m 21s) (2200 7%) 1.5146 3.66
8m 38s (- 104m 4s) (2300 7%) 1.5927 2.64
9m 2s (- 103m 56s) (2400 8%) 1.4729 2.45
9m 24s (-

70m 59s (- 39m 56s) (19200 64%) 1.0638 1.81
71m 19s (- 39m 32s) (19300 64%) 0.6304 1.34
71m 36s (- 39m 7s) (19400 64%) 1.5749 2.55
71m 59s (- 38m 45s) (19500 65%) 1.4901 3.95
72m 21s (- 38m 23s) (19600 65%) 0.8265 1.35
72m 40s (- 38m 0s) (19700 65%) 1.0061 1.86
73m 0s (- 37m 36s) (19800 66%) 1.3841 1.92
73m 18s (- 37m 12s) (19900 66%) 1.0478 1.55
73m 40s (- 36m 50s) (20000 66%) 1.1537 1.90
74m 2s (- 36m 28s) (20100 67%) 1.4716 2.65
74m 27s (- 36m 7s) (20200 67%) 0.8803 1.44
74m 44s (- 35m 42s) (20300 67%) 0.8392 1.51
75m 11s (- 35m 23s) (20400 68%) 1.4879 2.91
75m 37s (- 35m 2s) (20500 68%) 0.5382 1.35
76m 0s (- 34m 40s) (20600 68%) 1.5708 2.54
76m 26s (- 34m 20s) (20700 69%) 1.4106 2.36
76m 53s (- 34m 0s) (20800 69%) 1.2042 2.24
77m 14s (- 33m 37s) (20900 69%) 1.8850 2.92
77m 37s (- 33m 16s) (21000 70%) 1.3627 2.19
77m 58s (- 32m 53s) (21100 70%) 1.2007 2.12
78m 18s (- 32m 30s) (21200 70%) 1.6661 3.15
78m 40s (- 32m 8s) (21300 71%) 1.3094 2.74
79m 1s (- 31m 45s) (21400 71%) 0.9328 2.3

In [109]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_4.pth')

In [110]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [11989, 18, 23, 0, 28, 13, 16, 47, 65, 253, 380, 379, 1084, 792, 577, 1032, 704, 975, 410, 397, 177, 240, 15, 74, 240, 59, 83, 69, 26, 102, 6, 53, 17, 0, 1, 25, 17, 0, 0, 1, 5, 1, 0, 0, 0, 0, 2, 0, 3, 0, 0, 1, 0, 10, 27, 2, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 105, 0, 1, 2]


### 6.3.4 Teacher Forcing Ratio = 0.25 + Learning Rate harved + Random masking 35 % of corpus words


In [111]:
dialogues_Master_var_035 = variableFromCorpus(lang_M2DS, dialogues_Master, rand = 0.35)

In [112]:
chatbot.decoder.tf_ratio = 0.25

In [114]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_035, n_iters = 10000, per_dialogue = True, learning_rate=0.025)
trainer.train(chatbot, dialogues_Master_var_035, n_iters = 20000, per_dialogue = True, learning_rate=0.01)

0m 23s (- 38m 34s) (100 1%) 2.4771 3.65
0m 47s (- 38m 44s) (200 2%) 2.3943 3.72
1m 11s (- 38m 36s) (300 3%) 2.2756 4.23
1m 32s (- 36m 58s) (400 4%) 1.7730 4.85
1m 48s (- 34m 21s) (500 5%) 3.3875 6.24
2m 9s (- 33m 45s) (600 6%) 2.3651 4.46
2m 29s (- 33m 1s) (700 7%) 2.6537 4.27
2m 49s (- 32m 34s) (800 8%) 2.7192 4.42
3m 19s (- 33m 41s) (900 9%) 3.1017 5.22
3m 41s (- 33m 15s) (1000 10%) 2.5432 5.53
4m 0s (- 32m 22s) (1100 11%) 2.3270 3.66
4m 20s (- 31m 46s) (1200 12%) 2.3444 4.97
4m 43s (- 31m 39s) (1300 13%) 2.1564 3.76
5m 7s (- 31m 28s) (1400 14%) 1.6492 3.30
5m 28s (- 31m 3s) (1500 15%) 2.1249 3.79
5m 53s (- 30m 56s) (1600 16%) 2.3006 4.94
6m 13s (- 30m 25s) (1700 17%) 1.7348 2.61
6m 34s (- 29m 59s) (1800 18%) 2.3900 3.67
6m 58s (- 29m 44s) (1900 19%) 1.7744 3.18
7m 19s (- 29m 19s) (2000 20%) 2.4909 4.44
7m 40s (- 28m 54s) (2100 21%) 2.2540 4.43
8m 3s (- 28m 35s) (2200 22%) 2.6902 5.25
8m 26s (- 28m 14s) (2300 23%) 1.5523 3.32
8m 49s (- 27m 58s) (2400 24%) 1.7410 3.06
9m 12s (- 27m 37

35m 38s (- 38m 36s) (9600 48%) 1.8932 2.88
36m 2s (- 38m 16s) (9700 48%) 2.2780 4.13
36m 26s (- 37m 56s) (9800 49%) 1.9737 4.25
36m 49s (- 37m 34s) (9900 49%) 1.8094 4.89
37m 9s (- 37m 9s) (10000 50%) 2.4102 4.51
37m 32s (- 36m 47s) (10100 50%) 2.0602 3.82
37m 52s (- 36m 23s) (10200 51%) 1.9970 3.76
38m 14s (- 36m 1s) (10300 51%) 1.8063 3.65
38m 37s (- 35m 39s) (10400 52%) 1.7341 3.48
38m 59s (- 35m 17s) (10500 52%) 1.4530 2.94
39m 17s (- 34m 51s) (10600 53%) 1.7491 4.21
39m 37s (- 34m 26s) (10700 53%) 1.8249 3.68
39m 59s (- 34m 4s) (10800 54%) 1.6342 2.96
40m 15s (- 33m 36s) (10900 54%) 1.8190 3.29
40m 39s (- 33m 15s) (11000 55%) 1.5228 4.27
40m 59s (- 32m 51s) (11100 55%) 2.1728 4.77
41m 24s (- 32m 32s) (11200 56%) 2.2205 4.01
41m 51s (- 32m 13s) (11300 56%) 2.0870 3.07
42m 8s (- 31m 47s) (11400 56%) 1.7133 3.33
42m 28s (- 31m 23s) (11500 57%) 1.9162 4.57
42m 55s (- 31m 4s) (11600 57%) 1.3890 2.81
43m 19s (- 30m 44s) (11700 58%) 1.6627 3.50
43m 41s (- 30m 21s) (11800 59%) 2.0512 5.18

In [115]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_5.pth')

In [116]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse (19854 au total)

The repartition of errors : [12540, 23, 42, 1, 0, 7, 14, 70, 14, 165, 488, 347, 1045, 435, 796, 1230, 713, 827, 288, 337, 144, 222, 62, 38, 230, 49, 73, 61, 11, 54, 0, 0, 3, 3, 0, 22, 21, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 69, 0, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 105]


In [117]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var    , n_iters = 15000, per_dialogue = True, learning_rate=0.01)
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 15000, per_dialogue = True, learning_rate=0.01)
trainer.train(chatbot, dialogues_Master_var_035, n_iters = 15000, per_dialogue = True, learning_rate=0.01)

0m 20s (- 51m 9s) (100 0%) 0.5125 1.13
0m 38s (- 47m 56s) (200 1%) 0.5551 0.88
1m 1s (- 50m 17s) (300 2%) 0.1447 0.26
1m 24s (- 51m 10s) (400 2%) 0.3666 0.66
1m 48s (- 52m 28s) (500 3%) 0.5282 0.94
2m 9s (- 51m 40s) (600 4%) 0.1727 0.54
2m 31s (- 51m 41s) (700 4%) 0.4879 1.01
2m 55s (- 51m 59s) (800 5%) 0.3827 0.51
3m 18s (- 51m 57s) (900 6%) 0.4165 0.92
3m 35s (- 50m 17s) (1000 6%) 0.3615 0.96
3m 51s (- 48m 39s) (1100 7%) 0.5762 0.82
4m 10s (- 47m 59s) (1200 8%) 0.4916 1.01
4m 30s (- 47m 33s) (1300 8%) 0.6307 1.31
4m 54s (- 47m 42s) (1400 9%) 0.3065 0.56
5m 18s (- 47m 44s) (1500 10%) 0.2675 0.37
5m 42s (- 47m 48s) (1600 10%) 0.6217 0.90
6m 5s (- 47m 36s) (1700 11%) 0.3693 0.58
6m 25s (- 47m 10s) (1800 12%) 0.3370 0.65
6m 51s (- 47m 16s) (1900 12%) 0.6086 1.07
7m 12s (- 46m 52s) (2000 13%) 0.4904 0.95
7m 36s (- 46m 43s) (2100 14%) 0.2486 0.60
7m 56s (- 46m 14s) (2200 14%) 0.4753 0.88
8m 17s (- 45m 45s) (2300 15%) 0.4648 0.73
8m 40s (- 45m 32s) (2400 16%) 0.5669 0.96
9m 3s (- 45m 18s) (

16m 24s (- 38m 17s) (4500 30%) 0.9498 1.90
16m 43s (- 37m 47s) (4600 30%) 0.8257 1.40
17m 2s (- 37m 21s) (4700 31%) 1.2022 2.09
17m 28s (- 37m 8s) (4800 32%) 0.9018 1.40
17m 52s (- 36m 50s) (4900 32%) 1.4693 2.56
18m 18s (- 36m 37s) (5000 33%) 1.3343 3.21
18m 44s (- 36m 23s) (5100 34%) 1.9469 3.51
19m 10s (- 36m 8s) (5200 34%) 1.0857 2.01
19m 34s (- 35m 49s) (5300 35%) 1.1242 2.32
19m 55s (- 35m 24s) (5400 36%) 1.2846 2.31
20m 18s (- 35m 4s) (5500 36%) 1.7803 3.31
20m 35s (- 34m 34s) (5600 37%) 1.8366 2.99
21m 0s (- 34m 16s) (5700 38%) 1.3055 2.75
21m 21s (- 33m 53s) (5800 38%) 1.2298 3.21
21m 39s (- 33m 24s) (5900 39%) 1.9025 3.03
21m 59s (- 32m 59s) (6000 40%) 0.8506 1.40
22m 18s (- 32m 33s) (6100 40%) 1.0911 2.39
22m 40s (- 32m 11s) (6200 41%) 2.0062 3.15
22m 56s (- 31m 41s) (6300 42%) 1.2325 1.98
23m 18s (- 31m 19s) (6400 42%) 0.9966 1.56
23m 41s (- 30m 58s) (6500 43%) 0.6021 0.63
24m 1s (- 30m 34s) (6600 44%) 1.4569 3.33
24m 26s (- 30m 17s) (6700 44%) 1.7513 3.51
24m 48s (- 29m 55

32m 12s (- 22m 41s) (8800 58%) 1.2405 2.47
32m 32s (- 22m 18s) (8900 59%) 1.6941 3.25
32m 56s (- 21m 57s) (9000 60%) 2.1166 4.27
33m 20s (- 21m 37s) (9100 60%) 1.2547 2.51
33m 38s (- 21m 12s) (9200 61%) 1.4177 2.84
34m 0s (- 20m 50s) (9300 62%) 1.7841 3.83
34m 27s (- 20m 31s) (9400 62%) 2.2960 3.90
34m 46s (- 20m 8s) (9500 63%) 1.8453 4.09
35m 9s (- 19m 46s) (9600 64%) 1.8976 3.80
35m 33s (- 19m 25s) (9700 64%) 1.6298 4.01
35m 54s (- 19m 3s) (9800 65%) 1.2602 2.43
36m 17s (- 18m 41s) (9900 66%) 1.7530 3.64
36m 37s (- 18m 18s) (10000 66%) 1.7077 3.33
36m 59s (- 17m 56s) (10100 67%) 1.3849 3.10
37m 20s (- 17m 34s) (10200 68%) 0.6796 1.39
37m 43s (- 17m 12s) (10300 68%) 2.3482 4.97
37m 59s (- 16m 48s) (10400 69%) 1.3933 2.80
38m 19s (- 16m 25s) (10500 70%) 2.7315 4.79
38m 40s (- 16m 3s) (10600 70%) 1.0025 1.87
39m 3s (- 15m 41s) (10700 71%) 2.5156 5.25
39m 28s (- 15m 21s) (10800 72%) 1.4717 3.56
39m 49s (- 14m 58s) (10900 72%) 2.1600 3.70
40m 9s (- 14m 36s) (11000 73%) 1.7421 4.00
40m 36s

In [118]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_6.pth')

In [119]:
# 13082
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12533, 26, 9, 2, 0, 1, 13, 81, 183, 188, 483, 295, 1013, 655, 668, 1189, 539, 620, 428, 264, 198, 256, 8, 78, 324, 59, 71, 44, 11, 73, 0, 0, 2, 8, 0, 22, 19, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5, 0, 0, 0, 0, 66, 2, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 105]


## 6.4 test

### 6.4.1 Error counts



In [38]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12759, 0, 68, 0, 0, 29, 0, 96, 54, 113, 645, 395, 284, 591, 971, 902, 577, 1111, 136, 330, 143, 281, 0, 0, 67, 108, 47, 43, 1, 35, 0, 0, 0, 0, 0, 16, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35]


### 6.4.2 Visualization of attention

In [72]:
def showAnswer(model, dialogue):
    answer, log_probas, attn1_w, attn2_w, _ = model.answerTrain(dialogue)
    dialogue = model.flattenDialogue(dialogue)
    print('User : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-2][:-1]]))
    print('target : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-1][:-1]]))
    print('predic : ', ' '.join([model.lang.index2word[int(word)] for word in answer]))
    return

In [77]:
chatbot.eval()
n_errors = 14
i = 724
dialogue = dialogues_Master_var[i][:2]
#chatbot.showAttention(dialogue)
showAnswer(chatbot, dialogue)

User :  comment allez - vous ?
target :  je vais bien merci , en quoi puis - je vous aider ?
predic :  informatique ligne de quel daterentree ? ou mathematiques etes les UNK UNK quand UNK UNK UNK quand UNK UNK UNK quand UNK UNK UNK quand UNK UNK UNK quand UNK UNK quand UNK UNK UNK quand UNK UNK quand UNK UNK UNK quand UNK UNK quand UNK UNK UNK quand UNK UNK quand UNK UNK UNK quand UNK UNK quand UNK UNK UNK quand UNK UNK quand UNK UNK UNK quand UNK UNK quand UNK


In [53]:
chatbot.eval()
n_errors = 14
i = 724
dialogue = dialogues_Master_test_var[z[n_errors][i][0]][:z[n_errors][i][1]+1]
#chatbot.showAttention(dialogue)
showAnswer(chatbot, dialogue)

User :  oui , j ' ai etudie a unk_token
target :  etes - vous un etudiant inscrit a lyon 1 ?
predic :  il faut que vous deposiez votre candidature directement aupres de campus france .


In [37]:
chatbot.eval()
i = 10
dialogue = dialogues_Master_test_var[i][:-3]
chatbot.showAttention(dialogue)

ValueError: too many values to unpack (expected 4)

### 6.4.3 Interactive mode

write **quit** or **q** or **eoc** (end of conversation) to close conversation interface :

In [21]:
def ReplaceMotVar(motsVar, raw_sentence):
    sentence = []
    word_list = raw_sentence.split(' ')
    for word in word_list :
        if word in motsVar.keys() :
            sentence.append(motsVar[word])
        else :
            sentence.append(word)
    return ' '.join(sentence)


def repair(sentence) :
    s = re.sub(" ' ", "'", sentence)
    s = re.sub(" - ", "-", s)
    s = re.sub(" ,", ",", s)
    s = re.sub(r'(?<=\d) \. (?=\d)', '.', s)
    s = re.sub(" \.", ".", s)
    s = re.sub("\( ", "(", s)
    s = re.sub(" \)", ")", s)
    s = s[0].upper() + s[1:]
    return s


def InteractiveEvaluation(agent) :
    agent.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [128]:
chatbot.eval()
InteractiveEvaluation(chatbot)

comment on peut payer ?
Bot  : Vous pouvez payer par carte, par cheque bancaire, par virement ou en espece.
quels sont les modes de payement
Bot  : Le stage doit avoir une duree de 4 a 6 mois.
combien y a - t - il d ' ects par cours ?
Bot  : Il y a en tout 400h de cours.
combien d'etcs
Bot  : Il y a en tout 400h de cours.
le nombre d'etcs
Bot  : D'accord, comment puis-je vous aider ?
le nombre d etcs est de combien ?
Bot  : Je suis la pour vous renseigner sur le master 2 data science.
etcs
Bot  : Avez-vous d'autres questions ?
oui, est ce que c'est difficile ?
Bot  : Non, mais il faut travailler assidument.
ca veur dire qu'il faut travailler jours et nuits ?
Bot  : Il faut que vous deposiez votre candidature directement aupres de campus france.
quit


In [22]:
chatbot.eval()
InteractiveEvaluation(chatbot)

salut toi
Bot  : Bonjour, comment puis-je vous aider ?
comment tu te nomme ?
Bot  : Je suis la pour vous renseigner sur le master 2 data science.
quel est ton nom ?
Bot  : Je m'appelle Horace.
cool enchanté horace
Bot  : Bonjour, comment puis-je vous aider ?
what yu thking know
Bot  : Je vous en prie
ta gueule
Bot  : D'accord, comment puis-je vous aider ?
quit


# 7 Noise filtering

## 7.1 Noisy data generation

In [23]:
random.seed(42)

In [24]:
def randomMask(clean_sentences, rand = 0.2) :
    def choseWord(word) :
        return word if random.random() >= rand else 'UNK'
    masked_sentences = []
    for sentence in clean_sentences :
        words = [choseWord(word) for word in sentence.split()]
        masked_sentences.append(' '.join(words))
    return masked_sentences



def randomNoise(clean_sentences, lang, prop = 1.0):
    noisy_sentences = []
    for sentence in clean_sentences :
        if len(sentence.split()) >= 5 :
            words = [lang.index2word[random.randint(3, lang.n_words-1)] for _ in sentence.split()]
            words = [word for i, word in enumerate(words) if i < 5 or random.random() < prop]
            rand_unk = random.randint(1, len(words)-1)
            words.insert(rand_unk, 'UNK')
            if rand_unk > len(words)/ 2 : words += random.choice(['?', '.'])
            noisy_sentences.append(' '.join(words))
    return noisy_sentences

In [25]:
clean_sentences = [sentence for sentence in list(set([' '.join(pair[0]) for diag in dialogues_Master for pair in diag]))]
print(len(clean_sentences))

masked_sentences_000 = []
masked_sentences_015 = []
masked_sentences_025 = []
noisy_sentences_100 = []
noisy_sentences_050 = []

for _ in range(10) :
    masked_sentences_000 += clean_sentences
    masked_sentences_015 += randomMask(clean_sentences, rand = 0.15)
    masked_sentences_025 += randomMask(clean_sentences, rand = 0.25)

    noisy_sentences_100 += randomNoise(clean_sentences, lang_M2DS, prop = 1.0)
    noisy_sentences_050 += randomNoise(clean_sentences, lang_M2DS, prop = 0.5)

1364


In [26]:
discriminator_dataset = [[sentence, 1] for sentence in masked_sentences_000 + masked_sentences_015 + masked_sentences_025] + \
                        [[sentence, 0] for sentence in noisy_sentences_100 + noisy_sentences_050]
random.shuffle(discriminator_dataset)
len(discriminator_dataset)

66900

In [27]:
discriminator_dataset[7][0]

'les cours sont UNK memes entre UNK informatique UNK UNK ?'

## 7.2 Noise filtering model

In [28]:
from chatNLP.utils.Noise_Filter import NoiseFilter, NoiseFilterTrainer, NoiseFilterWrapper

In [29]:
noise_filter = NoiseFilter(chatbot, pretrained = True, layers = [50, 50]).to(device)

In [30]:
noise_filter.nbParametres()

10202

In [ ]:
#noise_filter.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_noise_filter.pth'))

In [34]:
noise_filter_trainer = NoiseFilterTrainer(device,
                                          criterion = nn.NLLLoss(), 
                                          optimizer = optim.SGD)

In [35]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 20000, learning_rate=0.01)

0m 1s (- 4m 30s) (100 0%) 0.7033 0.58
0m 2s (- 3m 36s) (200 1%) 0.6104 0.28
0m 3s (- 3m 17s) (300 1%) 0.6632 0.47
0m 3s (- 3m 7s) (400 2%) 0.5668 0.39
0m 4s (- 3m 1s) (500 2%) 0.6100 0.44
0m 5s (- 2m 56s) (600 3%) 0.6134 0.43
0m 6s (- 2m 52s) (700 3%) 0.5791 0.38
0m 7s (- 2m 49s) (800 4%) 0.5662 0.31
0m 7s (- 2m 47s) (900 4%) 0.5960 0.40
0m 8s (- 2m 45s) (1000 5%) 0.5095 0.29
0m 9s (- 2m 43s) (1100 5%) 0.4927 0.26
0m 10s (- 2m 41s) (1200 6%) 0.5237 0.32
0m 11s (- 2m 39s) (1300 6%) 0.4389 0.21
0m 11s (- 2m 38s) (1400 7%) 0.4833 0.31
0m 12s (- 2m 38s) (1500 7%) 0.3968 0.23
0m 13s (- 2m 36s) (1600 8%) 0.4835 0.28
0m 14s (- 2m 35s) (1700 8%) 0.4663 0.27
0m 15s (- 2m 34s) (1800 9%) 0.4843 0.26
0m 16s (- 2m 33s) (1900 9%) 0.4760 0.29
0m 16s (- 2m 31s) (2000 10%) 0.5011 0.31
0m 17s (- 2m 31s) (2100 10%) 0.4996 0.33
0m 18s (- 2m 32s) (2200 11%) 0.5167 0.30
0m 19s (- 2m 32s) (2300 11%) 0.5095 0.27
0m 20s (- 2m 31s) (2400 12%) 0.5487 0.33
0m 21s (- 2m 30s) (2500 12%) 0.4961 0.26
0m 22s (- 2m 28s

In [37]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.005)
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 30000, learning_rate=0.001)

0m 1s (- 1m 55s) (100 1%) 0.3162 0.15
0m 2s (- 1m 42s) (200 2%) 0.2730 0.18
0m 2s (- 1m 33s) (300 3%) 0.2949 0.19
0m 3s (- 1m 28s) (400 4%) 0.2548 0.16
0m 4s (- 1m 28s) (500 5%) 0.2382 0.16
0m 5s (- 1m 25s) (600 6%) 0.2973 0.14
0m 6s (- 1m 23s) (700 7%) 0.2080 0.12
0m 7s (- 1m 21s) (800 8%) 0.3292 0.17
0m 7s (- 1m 19s) (900 9%) 0.2538 0.14
0m 8s (- 1m 18s) (1000 10%) 0.4207 0.17
0m 9s (- 1m 16s) (1100 11%) 0.2301 0.10
0m 10s (- 1m 15s) (1200 12%) 0.3076 0.19
0m 11s (- 1m 14s) (1300 13%) 0.2710 0.17
0m 11s (- 1m 13s) (1400 14%) 0.3855 0.19
0m 12s (- 1m 11s) (1500 15%) 0.1809 0.08
0m 13s (- 1m 10s) (1600 16%) 0.1815 0.11
0m 14s (- 1m 9s) (1700 17%) 0.2196 0.14
0m 15s (- 1m 9s) (1800 18%) 0.3264 0.17
0m 16s (- 1m 8s) (1900 19%) 0.2728 0.17
0m 16s (- 1m 7s) (2000 20%) 0.2735 0.20
0m 17s (- 1m 6s) (2100 21%) 0.3328 0.16
0m 18s (- 1m 5s) (2200 22%) 0.2397 0.16
0m 19s (- 1m 4s) (2300 23%) 0.2542 0.16
0m 20s (- 1m 3s) (2400 24%) 0.2971 0.16
0m 20s (- 1m 2s) (2500 25%) 0.1961 0.14
0m 21s (- 1m 

1m 29s (- 2m 45s) (10500 35%) 0.2814 0.22
1m 30s (- 2m 44s) (10600 35%) 0.1586 0.10
1m 30s (- 2m 44s) (10700 35%) 0.2575 0.14
1m 31s (- 2m 43s) (10800 36%) 0.2443 0.15
1m 32s (- 2m 42s) (10900 36%) 0.2133 0.12
1m 33s (- 2m 41s) (11000 36%) 0.2571 0.13
1m 34s (- 2m 40s) (11100 37%) 0.3312 0.21
1m 34s (- 2m 39s) (11200 37%) 0.3057 0.20
1m 35s (- 2m 38s) (11300 37%) 0.1807 0.11
1m 36s (- 2m 37s) (11400 38%) 0.2187 0.15
1m 37s (- 2m 36s) (11500 38%) 0.2628 0.14
1m 38s (- 2m 35s) (11600 38%) 0.2337 0.12
1m 39s (- 2m 34s) (11700 39%) 0.2829 0.20
1m 39s (- 2m 34s) (11800 39%) 0.2000 0.15
1m 40s (- 2m 33s) (11900 39%) 0.2578 0.15
1m 41s (- 2m 32s) (12000 40%) 0.2358 0.14
1m 42s (- 2m 31s) (12100 40%) 0.2393 0.13
1m 43s (- 2m 30s) (12200 40%) 0.2687 0.17
1m 43s (- 2m 29s) (12300 41%) 0.2644 0.12
1m 44s (- 2m 28s) (12400 41%) 0.2023 0.13
1m 45s (- 2m 27s) (12500 41%) 0.2699 0.23
1m 46s (- 2m 26s) (12600 42%) 0.3215 0.17
1m 47s (- 2m 26s) (12700 42%) 0.3438 0.21
1m 48s (- 2m 25s) (12800 42%) 0.27

In [39]:
torch.save(noise_filter.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final noise_filter.pth')

## 7.3 Noise filtering integration

model built on top of a pretrained chatbot model

In [ ]:
def SmartInteractiveEvaluation(agent) :
    agent.chatbot.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [ ]:
noise_filter.eval()
smart_chatbot = NoiseFilterWrapper(noise_filter, chatbot)
smart_chatbot.eval()
SmartInteractiveEvaluation(smart_chatbot)